# Sequence to Sequence Learning Example
Original example is posted on the [Keras blog here](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html). This is a walkthrough to ensure I can implement similar models. 

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.recurrent import GRU, LSTM
import numpy as np

Using TensorFlow backend.


In [2]:
# Load Data
filename    = 'fra.txt'
batch_size  = 64      # Batch size for training.
epochs      = 100     # Number of epochs to train for.
latent_dim  = 256     # Latent dimensionality of the encoding space.
num_samples = 10000   # Number of samples to train on.

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(filename).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 63


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_target_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [5]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 16, 73)
(10000, 63, 92)
(10000, 63, 92)


## Training

### Encoder Model

In [17]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = GRU(latent_dim, return_state=True)
encoder_outputs, encoder_state = encoder(encoder_inputs)  # Discard `encoder_outputs` and only keep the states.

### Decoder Model

In [25]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_gru    = GRU(latent_dim, return_sequences=True,return_state=True)
decoder_outputs,_ = decoder_gru(decoder_inputs,initial_state=encoder_state)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, None, 73)      0                                            
____________________________________________________________________________________________________
input_10 (InputLayer)            (None, None, 92)      0                                            
____________________________________________________________________________________________________
gru_5 (GRU)                      [(None, 256), (None,  253440      input_6[0][0]                    
____________________________________________________________________________________________________
gru_8 (GRU)                      [(None, None, 256), ( 268032      input_10[0][0]                   
                                                                   gru_5[0][1]             

### Train Model

In [20]:
### Encoding
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=2,
          validation_split=0.2)

# model.save('s2s.h5')


Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 [==============================] - 51s - loss: 0.8749 - val_loss: 0.8318
Epoch 2/2
8000/8000 [==============================] - 49s - loss: 0.6270 - val_loss: 0.6908


## Inference
Next: inference mode (sampling). Here's the drill:
- encode input and retrieve initial decoder state
- run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
- Repeat with the current target token and current states


In [27]:
### Define Sampling Models
encoder_model = Model(encoder_inputs, encoder_state)

decoder_state_input = Input(shape=(latent_dim,))
decoder_outputs,decoder_state = decoder_gru(decoder_inputs, initial_state=decoder_state_input)

decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs, decoder_state_input], [decoder_outputs, decoder_state])

In [28]:
decoder_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, None, 92)      0                                            
____________________________________________________________________________________________________
input_11 (InputLayer)            (None, 256)           0                                            
____________________________________________________________________________________________________
gru_8 (GRU)                      [(None, None, 256), ( 268032      input_10[0][0]                   
                                                                   input_11[0][0]                   
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, None, 92)      23644       gru_8[1][0]             

In [21]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Decoding Sequence

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)      # Encode the input as state vectors.
    target_seq   = np.zeros((1, 1, num_decoder_tokens))    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)